In [38]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

In [39]:
# import zipfile
# data_dir = '/content/drive/MyDrive/dogs_and_cats.zip'
# zipfile.ZipFile(data_dir, 'r').extractall('/content/dogs_and_cats')

In [40]:
res = 128
train_dir = '/content/dogs_and_cats/dataset/training_set'
val_dir = '/content/dogs_and_cats/dataset/test_set'
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_data = data_gen.flow_from_directory(train_dir, target_size=(res, res), class_mode='binary')
val_data = data_gen.flow_from_directory(val_dir, target_size=(res, res), class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [41]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy')>0.95:
            print('Accuracy 85 reached')
            self.model.stop_training = True

callback = mycallback()

In [42]:
inn = tf.keras.Input(shape=(res,res,3))
Z1 = tfl.Conv2D(32, (3,3))(inn)
A1 = tfl.ReLU()(Z1)
P1 = tfl.MaxPool2D((2,2))(A1)
Z2 = tfl.Conv2D(32, (3,3))(P1)
A2 = tfl.ReLU()(Z2)
P2 = tfl.MaxPool2D((2,2))(A2)
Z3 = tfl.Conv2D(64, (3,3))(P2)
A3 = tfl.ReLU()(Z3)
P3 = tfl.MaxPool2D((2,2))(A3)
F = tfl.Flatten()(P3)
D1 = tfl.Dense(512, activation='relu')(F)
D2 = tfl.Dense(128, activation='relu')(D1)
D3 = tfl.Dense(64, activation='relu')(D2)
out = tfl.Dense(1, activation='sigmoid')(D3)

model = tf.keras.Model(inputs=inn, outputs=out)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [43]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 re_lu_12 (ReLU)             (None, 126, 126, 32)      0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 63, 63, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 61, 61, 32)        9248      
                                                                 
 re_lu_13 (ReLU)             (None, 61, 61, 32)        0         
                                                           

In [44]:
history = model.fit(train_data, batch_size=64, epochs=100, callbacks=[callback], validation_data=val_data)

Epoch 1/100
250/250 [==============================] - 27s 108ms/step - loss: 0.6463 - accuracy: 0.6091 - val_loss: 0.5688 - val_accuracy: 0.7050
Epoch 2/100
250/250 [==============================] - 27s 108ms/step - loss: 0.5437 - accuracy: 0.7234 - val_loss: 0.5530 - val_accuracy: 0.7100
Epoch 3/100
250/250 [==============================] - 26s 106ms/step - loss: 0.4631 - accuracy: 0.7803 - val_loss: 0.4936 - val_accuracy: 0.7650
Epoch 4/100
250/250 [==============================] - 27s 106ms/step - loss: 0.3879 - accuracy: 0.8282 - val_loss: 0.4897 - val_accuracy: 0.7745
Epoch 5/100
250/250 [==============================] - 27s 107ms/step - loss: 0.3160 - accuracy: 0.8622 - val_loss: 0.5285 - val_accuracy: 0.7870
Epoch 6/100
250/250 [==============================] - 28s 113ms/step - loss: 0.1938 - accuracy: 0.9227 - val_loss: 0.7032 - val_accuracy: 0.7500
Epoch 7/100
250/250 [==============================] - 26s 106ms/step - loss: 0.1060 - accuracy: 0.9567 - val_loss: 1.0248 -

In [45]:
# model.save('/content/drive/MyDrive/dog_vs_cat_cnn3.h5')